In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

### Data load

In [67]:
src_folder = os.path.join('data', 'idealista')

# df is created
ds = pd.DataFrame()

for file in os.listdir(src_folder):
    file = os.path.join(src_folder, file)
    ds = pd.concat([ds, pd.read_csv(file)], ignore_index = True).drop_duplicates()

ds = ds.reset_index(drop = True)
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           695 non-null    int64  
 1   title        695 non-null    object 
 2   n_rooms      675 non-null    float64
 3   size         695 non-null    int64  
 4   description  683 non-null    object 
 5   extra_info   689 non-null    object 
 6   image        682 non-null    object 
 7   img_no       695 non-null    int64  
 8   price        695 non-null    float64
 9   price_form   695 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 54.4+ KB


In [68]:
cols = ds.columns
cols_exclude = []

#convert int columns to float
for col in cols:
    if ds[col].dtype == 'int64':
        ds[col] = ds[col].astype('float64')
    elif ds[col].dtype == 'object':
        cols_exclude.append(col)

cols_model = [col for col in cols if col not in cols_exclude]

### Deal with NAs

In [69]:
ds['n_rooms'] = ds['n_rooms'].fillna(1)

### Data split

In [70]:
x = ds[cols_model].drop(['price'], axis=1)
y_original = ds['price']
y = y_original/ds['size']

# slit data into train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

### Model

In [71]:
model = LinearRegression()
model.fit(x_train, y_train)
r2_scored = model.score(x_test, y_test)
print("R2 Scored: ", r2_scored)

R2 Scored:  0.3170507097987837


### Evaluate model

y is price/m2

In [73]:
y_train_pred = model.predict(x_train)
rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
mae = mean_absolute_error(y_train, y_train_pred)
#print rmse and mae
print('Training RMSE score: {}.'.format(rmse))
print('Training MAE score: {}.\n'.format(mae))


y_test_pred = model.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
mae = mean_absolute_error(y_test, y_test_pred)  
print('Testing RMSE score: {}'.format(rmse))
print('Testing MAE score: {}'.format(mae))

Training RMSE score: 3.5387803016419754.
Training MAE score: 2.697483488391293.

Testing RMSE score: 4.072205955268407
Testing MAE score: 3.213118795415315


y is price

In [74]:
y_train_pred = model.predict(x_train)*x_train['size']
rmse = np.sqrt(mean_squared_error(y_original[y_train.index], y_train_pred))
mae = mean_absolute_error(y_original[y_train.index], y_train_pred)
#print rmse and mae
print('Training RMSE score: {}.'.format(rmse))
print('Training MAE score: {}.\n'.format(mae))


y_test_pred = model.predict(x_test)*x_test['size']
rmse = np.sqrt(mean_squared_error(y_original[y_test.index], y_test_pred))
mae = mean_absolute_error(y_original[y_test.index], y_test_pred)  
print('Testing RMSE score: {}'.format(rmse))
print('Testing MAE score: {}'.format(mae))

Training RMSE score: 513.7740335274872.
Training MAE score: 299.4289470220473.

Testing RMSE score: 528.4242340137963
Testing MAE score: 338.32187041631056
